# Mirrored Strategy: Basic

[Mirrored Strategy](https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy).

## Imports

In [ ]:
# Import TensorFlow and TensorFlow Datasets

import tensorflow_datasets as tfds
import tensorflow as tf
tfds.disable_progress_bar()

import os

Load the MNIST dataset and split it into training and test chunks.

In [ ]:
# Load the dataset we'll use for this lab
datasets, info = tfds.load(name='mnist', with_info=True, as_supervised=True, data_dir='./data')

mnist_train, mnist_test = datasets['train'], datasets['test']

Next,define `strategy` using the `MirroredStrategy()` class. Print to see the number of devices available.



In [ ]:
# Define the strategy to use and print the number of devices found
#solo imprime un dispositivo porque solo hay un gpu
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


Number of devices: 1


create the training and test examples, define the batch size and also define `BATCH_SIZE_PER_REPLICA` which is the distribution you are making for each available device.

In [ ]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
# Use for No Strategy
# BATCH_SIZE = BATCH_SIZE_PER_REPLICA * 1

A mapping function which normalizes your images:

In [ ]:
# Function for normalizing the image
def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255

    return image, label

Next, you create your training and evaluation datesets in the batch size you want by shuffling through your buffer size.

In [ ]:
# Set up the train and eval data set
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [ ]:
# Use for Mirrored Strategy -- comment out `with strategy.scope():` and deindent for no strategy
with strategy.scope():
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
    ])

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=12)

Epoch 1/12
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


938/938 [==============================] - 35s 37ms/step - loss: 0.1950 - accuracy: 0.9431
Epoch 2/12
938/938 [==============================] - 23s 25ms/step - loss: 0.0660 - accuracy: 0.9807
Epoch 3/12
938/938 [==============================] - 24s 25ms/step - loss: 0.0467 - accuracy: 0.9858
Epoch 4/12
938/938 [==============================] - 23s 25ms/step - loss: 0.0347 - accuracy: 0.9894
Epoch 5/12
938/938 [==============================] - 24s 25ms/step - loss: 0.0269 - accuracy: 0.9916
Epoch 6/12
938/938 [==============================] - 24s 25ms/step - loss: 0.0211 - accuracy: 0.9936
Epoch 7/12
938/938 [==============================] - 24s 25ms/step - loss: 0.0166 - accuracy: 0.9949
Epoch 8/12
938/938 [==============================] - 23s 25ms/step - loss: 0.0131 - accuracy: 0.9958
Epoch 9/12
938/938 [==============================] - 23s 25ms/step - loss: 0.0102 - accuracy: 0.9967
Epoch 10/12
938/938 [==============================] - 23s 25ms/step - loss: 0.0078 - accurac